In [2]:
import ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
import scipy
import neurokit2 as nk
import random
import pyhrv

In [8]:
def get_bvpfeatures(data, fs, index):
    #event_time = []
    features_bvp = None

    #for i in range(samples):
    #    event_time.append(int((i/samples)*len(data)))

    df, info = nk.ppg_process(data, sampling_rate=fs)

    # Visualize the processing
    #nk.ppg_plot(df, sampling_rate=fs)

    epochs = nk.epochs_create(df, events=[1],
                             epochs_start=-0.1, epochs_end=1.9)

    features_event = nk.ppg_eventrelated(epochs)
    #features = nk.ppg_intervalrelated(epochs)
    features = nk.ppg_analyze(df, sampling_rate=fs)

    features_event.reset_index(drop=True, inplace=True)

    features_bvp = pd.concat([features_event, features], axis = 1)
    features_bvp['index'] = index
    # features_bvp = features_bvp.dropna(axis='columns')
    return features_bvp

In [4]:
def bvp_prep(data):
    data = data.dropna(axis='columns')
    features = data
    features.replace([np.inf, -np.inf], np.nan, inplace=True)
    features = features.dropna(axis='columns')
    features.drop(columns=features.columns[:2], axis=1,  inplace=True)
    features = features.reset_index()
    return features

In [9]:
# a = get_bvpfeatures(bvp, 400, '1')
# a

,Label,Event_Onset,PPG_Rate_Baseline,PPG_Rate_Max,PPG_Rate_Min,PPG_Rate_Mean,PPG_Rate_SD,PPG_Rate_Max_Time,PPG_Rate_Min_Time,PPG_Rate_Trend_Linear,...,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC,index
0,1,1,92.28039,34.703737,-5.496823,3.383289,10.121064,0.431266,1.9,-1.582559,...,4.546594,1.135078,NaN,NaN,NaN,2.473939,1.934433,1.584745,1.627075,1
